<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

In [ ]:
from datasets import load_dataset

dataset_name = "Shushant/BiomedicalQuestionAnsweringDataset"
dataset = load_dataset(dataset_name, "train")
dataset = dataset["train"].train_test_split(test_size=3000, train_size=27000)

In [ ]:
import ast
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

def merge_answer(record):
  # convert string answer to dictionary
  answer = ast.literal_eval(record["answers"])["text"]
  record["answers"] = answer
  return record

# load T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
# the context summarization will be using T5 as explored previously
summarizer = AutoModelWithLMHead.from_pretrained('t5-small', return_dict=True)

def summarize_large_context(record):
  # the max_seq_len is 512, longer than that it will be automatically cut off
  if len(torch.split(record["context"], " ")) > 512:
    input_seq = tokenizer.encode("sumarize: " + record["context"], return_tensors="pt", max_length=512, truncation=True)
    summarized_seq = model.generate(input_seq, min_length = 100, max_length = 512)
    record["context"] = tokenizer.decode(summarized_seq[0])
  return record

def generate_text_field(record):
  record['text'] = "### Context: " + record['context'] + "### Question: " + record['question'] + "### Answer: " + record['answers']
  return record

training_dataset = dataset["train"]

training_dataset.add_column(name="text", column=["temp"]*len(training_dataset))

training_dataset = training_dataset.map(merge_answer)
# training_dataset = training_dataset.map(summarize_large_context)
training_dataset = training_dataset.map(generate_text_field, remove_columns=training_dataset.column_names)


In [ ]:
training_dataset["text"]

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [5]:
from peft import LoraConfig

lora_alpha = 16 # based on qlora paper
lora_dropout = 0.1 # for model up to 13B
lora_r = 64 # based on qlora paper

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


In [6]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 16
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3 # based on qlora paper
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)


In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)


In [8]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
!pip install wandb

In [12]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

wandb.init()

In [14]:
trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,8.225100
20,7.272600
30,6.774600
40,6.459100
50,6.147700
60,6.636000
70,6.355100
80,6.224200
90,6.119500
100,5.928700


KeyboardInterrupt: ignored